In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from iminuit import Minuit
from scipy import stats

In [2]:
Nbins = 30
xmin  = 9.35
xmax  = 10.05
binwidth = (xmax - xmin)/Nbins

In [3]:
data_columns = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
data_rows    = ['Results', 'Uncertainty']
data         = [[9.54, 9.36, 10.02, 9.87, 9.98, 9.86, 9.86, 9.81, 9.79],
                [0.15, 0.10, 0.11,  0.08, 0.14, 0.06, 0.03, 0.13, 0.04]]
df_data = pd.DataFrame(data = data, index = data_rows, columns = data_columns) #dataframe with data
df_data


,1,2,3,4,5,6,7,8,9
Results,9.54,9.36,10.02,9.87,9.98,9.86,9.86,9.81,9.79
Uncertainty,0.15,0.10,0.11,0.08,0.14,0.06,0.03,0.13,0.04


In [9]:
def gauss(x, mu, sigma):  #gaussian dist.
    return 1.0/(sigma * np.sqrt(2*np.pi)) * np.exp(-0.5 * (x - mu)**2 / sigma**2)

def fit_func(x, Ngauss, mu, sigma): #function to fit
    return Ngauss * binwidth * gauss(x, mu, sigma)

def z_score(x, mu, sigma):
    return (x - mu)/sigma
    
def chi2(Ngauss, mu, sigma):  #own chi2 calc
    y_fit = fit_func(x_test,Ngauss, mu, sigma)
    chi2  = np.sum(((y_test - y_fit) / ey_test)**2)
    return chi2

In [5]:
counts, bin_edges = np.histogram(data[0], bins = Nbins, range = (xmin, xmax))
#Histogram values(almost the same as plt.hist)
x_test  = (bin_edges[1:][counts>0] + bin_edges[:-1][counts>0])/2    #histogram, binning, errors
y_test  = counts[counts>0]
ey_test = np.sqrt(y_test)

student = [1, 2, 3, 4, 5, 6, 7, 8, 9]
xaxis   = np.linspace(9, 11, 100)

g_avg  = np.average(data[0])
g_wavg = np.average(data[0], weights = data[1]) #weighted average=best estimate of g
g_std  = np.std(data[0], ddof =1)
g_err  = g_std/np.sqrt(len(data[0]))   #best estimate of the uncertainty
print(f'Assuming indipendent measurements the best estimate of g is g = {g_wavg:.4f} +- {g_err:.4f}')

residual =[g_avg - data[0]]  
#Minuit fitting
minuit_chi2 = Minuit(chi2, Ngauss = Nbins, mu = g_wavg, sigma = g_std)
minuit_chi2.errordef = 1
minuit_chi2.migrad()

Assuming indipendent measurements the best estimate of g is g = 9.7742 +- 0.0700


<ipython-input-5-2958e361f4bd>:18: InitialParamWarning: errordef is not given, defaults to 1.
  minuit_chi2 = Minuit(chi2, Ngauss = Nbins, mu = g_wavg, sigma = g_std)


┌──────────────────────────────────┬──────────────────────────────────────┐
│ FCN = 0.4477                     │        Nfcn = 131 (131 total)        │
│ EDM = 1.41e-06 (Goal: 0.0002)    │                                      │
├───────────────┬──────────────────┼──────────────────────────────────────┤
│ Valid Minimum │ Valid Parameters │        No Parameters at limit        │
├───────────────┴──────────────────┼──────────────────────────────────────┤
│ Below EDM threshold (goal x 10)  │           Below call limit           │
├───────────────┬──────────────────┼───────────┬─────────────┬────────────┤
│   Hesse ok    │  Has Covariance  │ Accurate  │  Pos. def.  │ Not forced │
└───────────────┴──────────────────┴───────────┴─────────────┴────────────┘
┌───┬────────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name   │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼────────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ Ngauss │  0.08e3   │  0.30e3   │            │            │         │         │       │
│ 1 │ mu     │    9.8    │    0.9    │            │            │         │         │       │
│ 2 │ sigma  │    0.7    │    2.8    │            │            │         │         │       │
└───┴────────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘

In [10]:
Ngauss_fit, mu_fit, sigma_fit = minuit_chi2.values[:]
for name in minuit_chi2.parameters:
    print("Fit value of {0} = {1:.4f} +- {2:.4f}".format(name, minuit_chi2.values[name], minuit_chi2.errors[name]))
    
Chi2 = minuit_chi2.fmin.fval #get chi2 fit
Nvar = minuit_chi2.nfit      #number of variables used in the fit
Ndof = len(y_test) - Nvar    #degree.of.freedom

Prob = stats.chi2.sf(Chi2, Ndof)
print(f'Probability of doing worse P(chi2={Chi2:.4f}, Ndof={Ndof:.0f}) = {Prob:4.4f}')

Fit value of Ngauss = 83.5776 +- 305.2221
Fit value of mu = 9.7512 +- 0.9135
Fit value of sigma = 0.6966 +- 2.7897
Probability of doing worse P(chi2=0.4477, Ndof=5) = 0.9939


In [18]:
z_set  = []
P_Less = []
for x in data[0]:
    z = z_score(x, mu_fit, sigma_fit)
    z_set.append(z)
    P_less = stats.norm.cdf(z)
    P_Less.append(P_less)
    
print(f' Number of RMSE away from {mu_fit:.4f} : {z_set}') #not more than half a sigma away...everyone is ok i guess
print(f' The farest away are {z_set[1]:.4f} and {z_set[2]:.4f}')
#print(P_Less)



 Number of RMSE away from 9.7512 : [-0.30319927615504016, -0.5615915284727743, 0.3858467300255858, 0.17051985309413986, 0.3284262295105349, 0.15616472796537714, 0.15616472796537714, 0.08438910232156353, 0.05567885206403554]
 The farest away are -0.5616 and 0.3858
[0.3808690003065974, 0.28719717680281837, 0.6501949077487899, 0.5676993387548882, 0.6287052932101653, 0.5620484107065634, 0.5620484107065634, 0.5336264642665245, 0.5222011765145512]


In [23]:
g_real  = 9.8158
sg_real = 0.0001

agreement = z_score(mu_fit, g_real, sg_real)     #fit value wrt the real
Inv_agree = z_score(g_real, mu_fit, sigma_fit)   #real val wrt the fit
print(f'The estimated value is {agreement:.4f} real-sigma(={sg_real}) away from the real value of g = {g_real}')
print(f'The real value of g is {Inv_agree:.4f} sigma away from the best estimated value of g = {mu_fit:4f}')
#The best estimate si more than 600 sigma away from the real values, that is due to the haigh precision in the error of the real value
#and the poor dataset to begin with, on the other hand the real value is less than 0.1 sigma away from my estimate wich has a greater std

The estimated value is -645.8674 real-sigma(=0.0001) away from the real value of g = 9.8158
The real value of g is 0.0927 sigma away from the best estimated value of g = 9.751213


In [ ]:
fig, (ax, ax1, ax2) = plt.subplots(nrows = 3, ncols = 1)
hist = ax.hist(data[0], bins = Nbins, range=(xmin, xmax))
ax.errorbar(x_test, y_test, yerr=ey_test, fmt = '.', label='Values counts and error on bin')
ax.plot(xaxis, gauss(xaxis, g_wavg, g_std))
text = ax.set(xlabel = 'Values of g', ylabel = 'Counts', xlim=(9.0, 10.5))


ax1.errorbar(student, data[0], yerr = data[1], fmt ='.k', label='Measured value wiith error')
ax1.plot(student, [g_avg]*len(data[0]), '-', label='Mean value of g')
ax1.set(xlabel='Student', ylabel='Value of g')

hist2 = ax2.hist(residual)

ax.legend(loc = 'upper left')
ax1.legend(loc='best')